## Final Project Report

## Introduction

In this project, I will build a Natural Language Processing system to anaylze the attitude of movie comment is postive or nagative. The core of the system is Recurrent Neural Network(RNN). RNN is a class of artificial neural network where connection between nodes form a directed graph along a sequence. This is allows it to exhibit temporal dynamic behavior for a time sequence. Therefore, RNNS can use their internal state to process sequences of input. The input of NLP system is the text message, which is exactly one kind of sequnce information. So, I choose RNN as the core of this projcet's neural network. 

### Flowchat

The whole system could be divide into serveral parts. The first part is to convert the raw text-words into so-called tokens which are integer values. The integer values collect the input's word with the index of the a list of vocabulary. The second part is called embedding layer,  which are real-valued vectors, whose mapping will be trained along with the neural network, so as to map words with similar meanings to similar embedding-vectors. 

The third part is the core: input the embedding vectors to a RNN which can take sequences of arbitrary length as input and output a kind of summary of what it has seen in the input.This output is then squashed using a Sigmoid-function to give us a value between 0.0 and 1.0, where 0.0 is taken to mean a negative sentiment and 1.0 means a positive sentiment. 



<img src="./20_natural_language_flowchart.png" alt="Flowchart NLP" style="width: 300px;"/>

### Recurrent Neural Network

The basic building block in a Recurrent Neural Network (RNN) is a Recurrent Unit (RU). There are many different variants of recurrent units such as the rather clunky LSTM (Long-Short-Term-Memory) and the somewhat simpler GRU (Gated Recurrent Unit) which I will use in this project. Experiments in the literature suggest that the LSTM and GRU have roughly similar performance. Even simpler variants also exist and the literature suggests that they may perform even better than both LSTM and GRU, but they are not implemented in Keras which we will use in this tutorial.

The following figure shows the abstract idea of a recurrent unit, which has an internal state that is being updated every time the unit receives a new input. This internal state serves as a kind of memory. However, it is not a traditional kind of computer memory which stores bits that are either on or off. Instead the recurrent unit stores floating-point values in its memory-state, which are read and written using matrix-operations so the operations are all differentiable. This means the memory-state can store arbitrary floating-point values (although typically limited between -1.0 and 1.0) and the network can be trained like a normal neural network using Gradient Descent.

The new state-value depends on both the old state-value and the current input. For example, if the state-value has memorized that we have recently seen the word "not" and the current input is "good" then we need to store a new state-value that memorizes "not good" which indicates a negative sentiment.

The part of the recurrent unit that is responsible for mapping old state-values and inputs to the new state-value is called a gate, but it is really just a type of matrix-operation. There is another gate for calculating the output-values of the recurrent unit. The implementation of these gates vary for different types of recurrent units. This figure merely shows the abstract idea of a recurrent unit. The LSTM has more gates than the GRU but some of them are apparently redundant so they can be omitted.

In order to train the recurrent unit, we must gradually change the weight-matrices of the gates so the recurrent unit gives the desired output for an input sequence. This is done automatically in TensorFlow.


![Recurrent unit](images/20_recurrent_unit.png)

### Differernt number of Layer Unrolled Network

In this project, firstly I will use a Recurrent Neural Network with 3 recurrent units (or layers) denoted RU1, RU2 and RU3 in the "unrolled" figure below.

The first layer is much like the unrolled figure above for a single-layer RNN. First the recurrent unit RU1 has its internal state initialized to zero by Keras / TensorFlow. Then the word "this" is input to RU1 and it updates its internal state. Then it processes the next word "is", and so forth. But instead of outputting a single summary value at the end of the sequence, we use the output of RU1 for every time-step. This creates a new sequence that can then be used as input for the next recurrent unit RU2. The same process is repeated for the second layer and this creates a new output sequence which is then input to the third layer's recurrent unit RU3, whose final output is passed to a fully-connected Sigmoid layer that outputs a value between 0.0 (negative sentiment) and 1.0 (positive sentiment).

I will change different number of layers to see its influence.

![Unrolled 3-layer network](images/20_unrolled_3layers_flowchart.png)



## Imports

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

In [0]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [6]:
tf.__version__

'1.12.0'

In [7]:
tf.keras.__version__

'2.1.6-tf'

## Load Data

The whole dataset is 50000 movie comments from IMDB. The data is exactly the comment wirtten by audiences. The label is the 0 or 1. 0 means negative comment and 1 means positive comment. 

In [0]:
import imdb

In [9]:
imdb.maybe_download_and_extract()

- Download progress: 100.0%
Download finished. Extracting files.
Done.


In [0]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

Reset the size of train and test set. The number of training set is 45000. The number of testing set is 5000.

In [0]:
x_train_text = x_train_text + x_test_text[0:20000]
y_train =  y_train + y_test[0:20000]

In [0]:
x_test_text = x_test_text[20000:25001]
y_test =y_test[20000:25001]

In [13]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  45000
Test-set size:   5000


In [14]:
print("Train-set size: ", len(y_train))
print("Test-set size:  ", len(y_test))

Train-set size:  45000
Test-set size:   5000


In [0]:
#x_train_text[45000]

In [0]:
#y_train[24990:25010]

In [0]:
data_text = x_train_text + x_test_text

Let's find out what's the dataset looks like:

In [18]:
x_train_text[1]

'I was pleasantly surprised I quite liked this movie. Witty writing (some "inside" jokes I got, others I didn\'t - maybe due to actors speaking on top of one another), great acting (notably John Cassini), great cameos, interesting and unique directing. I rented it to see Jeffrey Meek (very disappointed he was in it such a short time, blink and you\'ll miss him!) but found the movie remarkably entertaining. I\'ll actually watch it again before I send back to Netflix. I think actors and wanna-be actors will thoroughly enjoy this movie. The ending is somewhat expected but wish they\'d done something different (and more positive). Too bad the movie wasn\'t better received except for in the "festival" market. I suggest it to anyone who loves the acting biz.'

In [19]:
y_train[1]

1.0

## Tokenizer


Since the neural network cannot directly deal with string type data. We need to use Tokenizer to change input from string type to integer type. We could build a collection between words with a index of dictionary by Toenizer. Here we set num_words =10000, which means we choose 10000 most popular words from the data-set.

In [0]:
num_words = 10000

In [0]:
tokenizer = Tokenizer(num_words=num_words)

In [22]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 11.6 s, sys: 106 ms, total: 11.7 s
Wall time: 11.7 s


In [0]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [24]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

In [0]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

one example from training set data and its integer type.

In [26]:
x_train_text[1]

'I was pleasantly surprised I quite liked this movie. Witty writing (some "inside" jokes I got, others I didn\'t - maybe due to actors speaking on top of one another), great acting (notably John Cassini), great cameos, interesting and unique directing. I rented it to see Jeffrey Meek (very disappointed he was in it such a short time, blink and you\'ll miss him!) but found the movie remarkably entertaining. I\'ll actually watch it again before I send back to Netflix. I think actors and wanna-be actors will thoroughly enjoy this movie. The ending is somewhat expected but wish they\'d done something different (and more positive). Too bad the movie wasn\'t better received except for in the "festival" market. I suggest it to anyone who loves the acting biz.'

In [27]:
np.array(x_train_tokens[1])

array([  10,   13, 3732,  760,   10,  179,  425,   11,   17, 1915,  490,
         47,  939,  616,   10,  189,  400,   10,  154,  273,  693,    5,
        150, 1387,   20,  342,    4,   27,  156,   78,  113, 3847,  315,
         78, 3437,  218,    2,  959,  975,   10, 1567,    9,    5,   63,
       4423, 9842,   52,  680,   28,   13,    8,    9,  138,    3,  334,
         55, 7833,    2,  474,  698,   87,   18,  255,    1,   17, 4557,
        427,  657,  160,  103,    9,  172,  159,   10, 2217,  141,    5,
       6514,   10,  101,  150,    2, 2930,   26,  150,   80, 1647,  354,
         11,   17,    1,  272,    6,  672,  862,   18,  641, 3397,  221,
        137,  279,    2,   51, 1161,   96,   74,    1,   17,  292,  126,
       2015,  556,   15,    8,    1, 1395, 2463,   10, 1472,    9,    5,
        250,   35, 1357,    1,  113])

We also need to convert the texts in the test-set to tokens.

In [0]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

## Padding and Truncating Data

The Recurrent Neural Network can take sequences of arbitrary length as input, but in order to use a whole batch of data, the sequences need to have the same length. 

I will use a sequence-length that covers most sequences in the data-set, and we will then truncate longer sequences and pad shorter sequences.

First we count the number of tokens in all the sequences in the data-set.

In [0]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [30]:
np.mean(num_tokens)

221.27716

In [31]:
np.max(num_tokens)

2209

In [32]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

544

In [33]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94532

In [0]:
pad = 'pre'

In [0]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [0]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [37]:
x_train_pad.shape

(45000, 544)

In [38]:
x_test_pad.shape

(5000, 544)

In [39]:
np.array(x_train_tokens[1])

array([  10,   13, 3732,  760,   10,  179,  425,   11,   17, 1915,  490,
         47,  939,  616,   10,  189,  400,   10,  154,  273,  693,    5,
        150, 1387,   20,  342,    4,   27,  156,   78,  113, 3847,  315,
         78, 3437,  218,    2,  959,  975,   10, 1567,    9,    5,   63,
       4423, 9842,   52,  680,   28,   13,    8,    9,  138,    3,  334,
         55, 7833,    2,  474,  698,   87,   18,  255,    1,   17, 4557,
        427,  657,  160,  103,    9,  172,  159,   10, 2217,  141,    5,
       6514,   10,  101,  150,    2, 2930,   26,  150,   80, 1647,  354,
         11,   17,    1,  272,    6,  672,  862,   18,  641, 3397,  221,
        137,  279,    2,   51, 1161,   96,   74,    1,   17,  292,  126,
       2015,  556,   15,    8,    1, 1395, 2463,   10, 1472,    9,    5,
        250,   35, 1357,    1,  113])

In [40]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## Tokenizer Inverse Map


For some strange reason, the Keras implementation of a tokenizer does not seem to have the inverse mapping from integer-tokens back to words, which is needed to reconstruct text-strings from lists of tokens. So we make that mapping here.

In [0]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [0]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

## Create the Recurrent Neural Network

Using the Keras API to create RNN

In [0]:
model = Sequential()

The first layer in the RNN is a so-called Embedding-layer which converts each integer-token into a vector of values. This is necessary because the integer-tokens may take on values between 0 and 10000 for a vocabulary of 10000 words. The RNN cannot work on values in such a wide range. The embedding-layer is trained as a part of the RNN and will learn to map words with similar semantic meanings to similar embedding-vectors, as will be shown further below

In [0]:
embedding_size = 8

The embedding-layer also needs to know the number of words in the vocabulary (num_words) and the length of the padded token-sequences (max_tokens). 

In [0]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

Set three GRU layers:

In [0]:
model.add(GRU(units=16, return_sequences=True))

In [0]:
model.add(GRU(units=8, return_sequences=True))

In [0]:
model.add(GRU(units=4))

The output of the third GRU layer will be the input of a dense-layer. The output of the dense-layer will be one integer number between 0 and 1:

In [0]:
model.add(Dense(1, activation='sigmoid'))

Use the Adam optimizer with the given learning-rate.

In [0]:
optimizer = Adam(lr=1e-3)

Compile the Keras model so it is ready for training.

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 544, 16)           1200      
_________________________________________________________________
gru_1 (GRU)                  (None, 544, 8)            600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2         
Total params: 81,963
Trainable params: 81,963
Non-trainable params: 0
_________________________________________________________________


## Train the Recurrent Neural Network

Now we could train the model. We set 5% of training set as a validation-set.

In [0]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42750 samples, validate on 2250 samples
Epoch 1/3
42750/42750 [==============================] - 475s 11ms/step - loss: 0.6872 - acc: 0.5644 - val_loss: 0.8404 - val_acc: 0.0000e+00
Epoch 2/3
42750/42750 [==============================] - 470s 11ms/step - loss: 0.6789 - acc: 0.5848 - val_loss: 0.8741 - val_acc: 0.0000e+00
Epoch 3/3
42750/42750 [==============================] - 462s 11ms/step - loss: 0.6587 - acc: 0.6307 - val_loss: 0.6130 - val_acc: 0.7444
CPU times: user 42min 26s, sys: 2min 19s, total: 44min 45s
Wall time: 23min 29s


In [0]:
%%time
result = model.evaluate(x_test_pad, y_test)

5000/5000 [==============================] - 24s 5ms/step
CPU times: user 42.7 s, sys: 2.42 s, total: 45.1 s
Wall time: 24.2 s


In [0]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 87.72%


## Try different neural network architecture

### Changing number of GRU layers

Here we will try 4 layers GRU network:

In [0]:
model2 = Sequential()
embedding_size = 8

model2.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

model2.add(GRU(units=32, return_sequences=True))
model2.add(GRU(units=16, return_sequences=True))
model2.add(GRU(units=8, return_sequences=True))
model2.add(GRU(units=4))
model2.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)

model2.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_10 (GRU)                 (None, 544, 32)           3936      
_________________________________________________________________
gru_11 (GRU)                 (None, 544, 16)           2352      
_________________________________________________________________
gru_12 (GRU)                 (None, 544, 8)            600       
_________________________________________________________________
gru_13 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 87,049
Trainable params: 87,049
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
model2.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42750 samples, validate on 2250 samples
Epoch 1/3
42750/42750 [==============================] - 853s 20ms/step - loss: 0.6044 - acc: 0.6593 - val_loss: 0.5591 - val_acc: 0.7520
Epoch 2/3
42750/42750 [==============================] - 843s 20ms/step - loss: 0.3477 - acc: 0.8570 - val_loss: 0.4354 - val_acc: 0.8129
Epoch 3/3
42750/42750 [==============================] - 847s 20ms/step - loss: 0.2624 - acc: 0.8983 - val_loss: 0.3618 - val_acc: 0.8560
CPU times: user 1h 16min 38s, sys: 4min 25s, total: 1h 21min 3s
Wall time: 42min 26s


In [0]:
%%time
result2 = model2.evaluate(x_test_pad, y_test)

5000/5000 [==============================] - 35s 7ms/step
CPU times: user 1min 2s, sys: 3.62 s, total: 1min 5s
Wall time: 35 s


In [0]:
print("Accuracy: {0:.2%}".format(result2[1]))

Accuracy: 86.14%


Here we will try 5 layers GRU network:

In [0]:
model3 = Sequential()
embedding_size = 8

model3.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

model3.add(GRU(units=64, return_sequences=True))
model3.add(GRU(units=32, return_sequences=True))
model3.add(GRU(units=16, return_sequences=True))
model3.add(GRU(units=8, return_sequences=True))
model3.add(GRU(units=4))
model3.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)

model3.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_14 (GRU)                 (None, 544, 64)           14016     
_________________________________________________________________
gru_15 (GRU)                 (None, 544, 32)           9312      
_________________________________________________________________
gru_16 (GRU)                 (None, 544, 16)           2352      
_________________________________________________________________
gru_17 (GRU)                 (None, 544, 8)            600       
_________________________________________________________________
gru_18 (GRU)                 (None, 4)                 156       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 5         
Total para

In [0]:
%%time
model3.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42750 samples, validate on 2250 samples
Epoch 1/3
42750/42750 [==============================] - 1309s 31ms/step - loss: 0.6794 - acc: 0.5859 - val_loss: 0.8985 - val_acc: 0.0053
Epoch 2/3
42750/42750 [==============================] - 1288s 30ms/step - loss: 0.6785 - acc: 0.5860 - val_loss: 0.9221 - val_acc: 0.0036
Epoch 3/3
42750/42750 [==============================] - 1286s 30ms/step - loss: 0.6789 - acc: 0.5843 - val_loss: 0.8776 - val_acc: 0.0000e+00
CPU times: user 1h 57min 16s, sys: 7min 11s, total: 2h 4min 28s
Wall time: 1h 4min 47s


In [0]:
%%time
result3 = model3.evaluate(x_test_pad, y_test)

5000/5000 [==============================] - 47s 9ms/step
CPU times: user 1min 25s, sys: 3.35 s, total: 1min 29s
Wall time: 47 s


In [0]:
print("Accuracy: {0:.2%}".format(result3[1]))

Accuracy: 0.00%


Now let's reduce the GRU layer number to 2

In [0]:
model4 = Sequential()
embedding_size = 8

model4.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))


model4.add(GRU(units=8, return_sequences=True))
model4.add(GRU(units=4))
model4.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)

model4.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [47]:
model4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 544, 8)            408       
_________________________________________________________________
gru_1 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 80,569
Trainable params: 80,569
Non-trainable params: 0
_________________________________________________________________


In [48]:
%%time
model4.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42750 samples, validate on 2250 samples
Epoch 1/3
42750/42750 [==============================] - 358s 8ms/step - loss: 0.4908 - acc: 0.7551 - val_loss: 0.6007 - val_acc: 0.7640
Epoch 2/3
42750/42750 [==============================] - 357s 8ms/step - loss: 0.3084 - acc: 0.8786 - val_loss: 0.3695 - val_acc: 0.8449
Epoch 3/3
42750/42750 [==============================] - 358s 8ms/step - loss: 0.2520 - acc: 0.9058 - val_loss: 0.4001 - val_acc: 0.8382
CPU times: user 31min 44s, sys: 2min 19s, total: 34min 4s
Wall time: 17min 55s


In [0]:
%%time
result4 = model4.evaluate(x_test_pad, y_test)

5000/5000 [==============================] - 16s 3ms/step
CPU times: user 28.1 s, sys: 1.74 s, total: 29.8 s
Wall time: 16.1 s


In [0]:
print("Accuracy: {0:.2%}".format(result4[1]))

Accuracy: 89.14%


###  Conclusion

In this section, I have try 4 kinds of number of GRU layers.: 2, 3, 4, 5. Increasing the number may not increase the accuarcy of the neural network. When number of layers increases to some certain numbers. it will cause gradient explosion. (When the number of GRU layers is 5, the accuarcy on test data becomes 0).

### Changing the value of batch size

#### Decrease the batch size to 32

In [0]:
model5 = Sequential()
embedding_size = 8

model5.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))


model5.add(GRU(units=8, return_sequences=True))
model5.add(GRU(units=4))
model5.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)

model5.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_3 (GRU)                  (None, 544, 8)            408       
_________________________________________________________________
gru_4 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 80,569
Trainable params: 80,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
model5.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=32)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42750 samples, validate on 2250 samples
Epoch 1/3
42750/42750 [==============================] - 581s 14ms/step - loss: 0.4445 - acc: 0.7863 - val_loss: 0.7254 - val_acc: 0.6920
Epoch 2/3
42750/42750 [==============================] - 610s 14ms/step - loss: 0.2854 - acc: 0.8883 - val_loss: 0.4382 - val_acc: 0.8160
Epoch 3/3
42750/42750 [==============================] - 592s 14ms/step - loss: 0.2340 - acc: 0.9121 - val_loss: 0.2270 - val_acc: 0.9147
CPU times: user 53min 24s, sys: 3min 12s, total: 56min 36s
Wall time: 29min 44s


In [0]:
%%time
result5 = model5.evaluate(x_test_pad, y_test)

5000/5000 [==============================] - 14s 3ms/step
CPU times: user 24.6 s, sys: 1.54 s, total: 26.1 s
Wall time: 14.2 s


In [0]:
print("Accuracy: {0:.2%}".format(result5[1]))

Accuracy: 91.98%


#### Increase the batch size to 128

In [0]:
model6 = Sequential()
embedding_size = 8

model6.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))


model6.add(GRU(units=8, return_sequences=True))
model6.add(GRU(units=4))
model6.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)

model6.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [0]:
model6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_5 (GRU)                  (None, 544, 8)            408       
_________________________________________________________________
gru_6 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 80,569
Trainable params: 80,569
Non-trainable params: 0
_________________________________________________________________


In [0]:
%%time
model6.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=128)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 42750 samples, validate on 2250 samples
Epoch 1/3
42750/42750 [==============================] - 170s 4ms/step - loss: 0.5245 - acc: 0.7257 - val_loss: 0.4876 - val_acc: 0.8236
Epoch 2/3
42750/42750 [==============================] - 168s 4ms/step - loss: 0.3249 - acc: 0.8717 - val_loss: 0.4835 - val_acc: 0.8040
Epoch 3/3
42750/42750 [==============================] - 165s 4ms/step - loss: 0.2670 - acc: 0.9000 - val_loss: 0.4796 - val_acc: 0.8133
CPU times: user 15min 8s, sys: 48.3 s, total: 15min 56s
Wall time: 8min 24s


In [0]:
%%time
result6 = model6.evaluate(x_test_pad, y_test)

5000/5000 [==============================] - 14s 3ms/step
CPU times: user 24.6 s, sys: 1.58 s, total: 26.2 s
Wall time: 14.2 s


In [0]:
print("Accuracy: {0:.2%}".format(result6[1]))

Accuracy: 80.52%


#### Conclusion

The influence of batch size is mainly on the train speed. Like the result we get: Increasing the batch size will speed up the training process, bu may cause decreasing the accuracy.